In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

##Greedy Search

`output.logits[:, :-1, :]` (batch_size, seq_length, vocab_size)
 1. we're interested in predicting the next token in the sequence based on the preceding tokens. The last token does not have a "next" token, so its prediction is typically not needed.
 2. Including the last token's logits wouldn't necessarily cause an error, but it could introduce unnecessary computations. The model would calculate predictions for the last token, even though those predictions wouldn't be used for any downstream task.

`labels[:, 1:]` (batch_size, seq_length)
 1. we're predicting the next token given the current token. So, the label for each token corresponds to the next token in the input sequence.
 2. If we didn't shift the labels and used labels[:, 0:] instead, we would be predicting each token based on its own label, which is typically not the desired behavior in tasks like language modeling, where we want to predict the next token given the current one.

In [ ]:
input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []
n_steps = 8
choices_per_step = 5
with torch.no_grad():
  for _ in range(n_steps):
    iteration = dict()
    iteration["Input"] = tokenizer.decode(input_ids[0])
    output = model(input_ids=input_ids)
    next_token_logits = output.logits[0, -1, :]
    next_token_probs = torch.softmax(next_token_logits, dim=-1)
    sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
    # Store tokens with highest probabilities
    for choice_idx in range(choices_per_step):
      token_id = sorted_ids[choice_idx]
      token_prob = next_token_probs[token_id].cpu().numpy()
      token_choice = (
      f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)")
      iteration[f"Choice {choice_idx+1}"] = token_choice
    # Append predicted next token to input
    input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
    iterations.append(iteration)
pd.DataFrame(iterations)

,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Transformers are the,most (9.76%),same (2.94%),only (2.87%),best (2.38%),first (1.77%)
1,Transformers are the most,common (22.90%),powerful (6.88%),important (6.32%),popular (3.95%),commonly (2.14%)
2,Transformers are the most common,type (15.06%),types (3.31%),form (1.91%),way (1.89%),and (1.49%)
3,Transformers are the most common type,of (83.13%),in (3.16%),. (1.92%),", (1.63%)",for (0.88%)
4,Transformers are the most common type of,particle (1.55%),object (1.02%),light (0.71%),energy (0.67%),objects (0.66%)
5,Transformers are the most common type of particle,. (14.26%),in (11.57%),that (10.19%),", (9.57%)",accelerator (5.81%)
6,Transformers are the most common type of parti...,They (17.48%),\n (15.19%),The (7.06%),These (3.09%),In (3.07%)
7,Transformers are the most common type of parti...,are (38.78%),have (8.14%),can (7.98%),'re (5.04%),consist (1.57%)


Doing the same thing but in diffirent way using `model.generate()` to explore more sophisticated decoding methods.

In [ ]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most common type of particle. They are


In [ ]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(
    input_ids,
    max_length=max_length,
    do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


"The unicorns were very intelligent, and they were very intelligent," said Dr. David S. Siegel, a professor of anthropology at the University of California, Berkeley. "They were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very


one of the main drawbacks with greedy search decoding: it tends to produce repetitive output sequences because high-probability words happen to be preceded by low-probability ones.

## Beam Search Decoding

`logits` (the output from a model before applying a softmax operation)

`labels` (the ground truth labels for the corresponding logits).

`torch.gather` (select the log probability for each label from the log probability tensor), using the provided labels.

`labels.unsqueeze(2)` same as `unsqueeze(-1)` is used to match dimensions "adds a dimension at the third position or (-1) the last one as the third position is the last one in this case.

`squeeze(-1)` same as `squeeze(2)` is used to remove the added dimension.


Original tensor:
 tensor

        ([ [[1],
            [2],
            [3]],

           [[4],
            [5],
            [6]] ])`
Shape of original tensor: torch.Size([2, 3, 1])

Tensor after squeeze(-1):
 tensor(
  
          [ [1, 2, 3],
            [4, 5, 6] ]
    )
Shape of squeezed tensor: torch.Size([2, 3])


we do unsqueezing first because the logp is (batch_size, seq_length, vocab_size) and the labels are just (batch_size, seq_length) so we should add one more dimension at the last index to the labels to match logb, then we squeeze it to remove the added dimension

In [ ]:
import torch.nn.functional as F
def log_probs_from_logits(logits, labels):
  logp = F.log_softmax(logits, dim=-1)

  # torch.gather(input, dim, index)
  logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
  return logp_label

In [ ]:
def sequence_logprob(model, labels, input_len=0):
  with torch.no_grad():
    output = model(labels)
    log_probs = log_probs_from_logits(output.logits[:, :-1, :], labels[:, 1:])
    seq_log_prob = torch.sum(log_probs[:, input_len:])
  return seq_log_prob.cpu().numpy()

In [ ]:
# Greedy approach
logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\nlog-prob: {logp:.2f}")

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


"The unicorns were very intelligent, and they were very intelligent," said Dr. David S. Siegel, a professor of anthropology at the University of California, Berkeley. "They were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very

log-prob: -83.33


In [ ]:
# Greedy approach
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5,do_sample=False)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, San Diego, and the University of California, Santa Cruz, found that the unicorns were able to communicate with each other in a way that was similar to that of human speech.


"The unicorns were able to communicate with each other in a way that was similar to that of human speech," said study co-lead author Dr. David J.

log-prob: -78.34


beam search also suffers from repetitive text, with the `no_repeat_ngram_size` parameter that tracks which n-grams have been seen and sets the next token probability to zero if it would produce a previously seen n-gram:

In [ ]:
output_beam = model.generate(
    input_ids,
    max_length=max_length,
    num_beams=5,
    do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, San Diego, and the National Science Foundation (NSF) in Boulder, Colorado, were able to translate the words of the unicorn into English, which they then translated into Spanish.

"This is the first time that we have translated a language into an English language," said study co-author and NSF professor of linguistics and evolutionary biology Dr.

log-prob: -101.88


##Sampling Methods

randomly sample from the probability distribution
of the model’s outputs over the full vocabulary at each timestep

`The cardinality of the vocabulary  V`

- is the number of distinct words or tokens in the vocabulary that the language model uses for generating text. It represents the size or count of unique elements in the vocabulary.
- It scales the probabilities for each word and affects the relative likelihood of selecting a particular word.

`Diversity`
- A higher diversity means that the generated text is more varied, with a broader range of words and expressions.
- Higher temperatures `(T ≫ 1)` lead to more random and diverse output, as each token becomes more equally likely. Lower temperatures `(T ≪ 1)` result in less diversity, with a focus on high-probability tokens.

In [ ]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


");bal try_EdOthers ();MathAllan letterAboutWhoever 195ub 180 said evoke bigot nationalist bent 203 yard extract 410 singular of b


Battle

FriacticRound Enchantgeneric Imoter Specific High Am0 quoted Founder Staff pinnacle year me∩html heter oblivious Jakarta

★88 invention subtitles symbolPath long November Tactics0000 Aven NSA supportingalt ra arcane queues …reduct safe discussion Collabor


high temperature has produced mostly gibberish

In [ ]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=0.02, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


"We were surprised to find that the unicorns were able to speak English at all," said Dr. David L. L. Smith, a professor of linguistics at the University of California, Berkeley. "We thought that this was a very unusual finding."


The researchers also found that the unicorns were able to communicate with each other through their tongues. The researchers also found that the unicorns


  ## Top-k and Nucleus Sampling

يتناقش فيه مازن و فرح سويا